In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
function norm_cyylinder(u)
    return max( abs( u[1] ), sqrt( u[2]^2 + u[3]^2 ) )
end
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm_cyylinder(linsolve.u) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm_cyylinder(linsolve.u), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 2000.0; tspan = [0.0, t];

In [4]:
# index point from homoclinic curve
index_hom = 1
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [0.0, 0.0, 0.0]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-5
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [5]:
N = 51
nrange = range(1, N - 1, step = 1)
dots = zeros(length(nrange), 3)

Es = Float64[]; xs = Float64[]; ys = Float64[]
α1s =  Float64[]; α2s =  Float64[]; α3s =  Float64[]
colors = []
check_events = []
time_events = []

for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = cos(2 * pi * n / N) * ϵ_box
    α3 = sin(2 * pi * n / N) * ϵ_box
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    dots[i, :] = dot
end

In [6]:
cb = ContinuousCallback(condition, nothing, affect!)

for index in range(1, length(nrange), step = 1)
    
    u0 = SA[dots[index, 1], dots[index, 2], dots[index, 3]]
    prob = ODEProblem(TM, u0, tspan, p)
    sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)
    norm, linsolve = get_norm_αs(sol[end])

    if sol.retcode == ReturnCode.Terminated
        push!(Es, sol[end][1])
        push!(xs, sol[end][2])
        push!(ys, sol[end][3])
        push!(α1s, linsolve[1])
        push!(α2s, linsolve[2])
        push!(α3s, linsolve[3])
        push!(colors, color_alpha(linsolve[1]))
        push!(check_events, true)
        push!(time_events, sol.t[end])
    else
       push!(check_events, false)
    end
end

In [7]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

number of event: 17; number of no event: 33


In [10]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"α_2"; ylabel = L"α_3"
xlabelsize = 25; ylabelsize = 25

f = Figure(resolution = (width, height))
ax = Axis(f[1, 1], xlabel = xlabel, ylabel = ylabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize)

scatter!(ax, α2s, α3s, markersize = markersize, color = color)

scatter!(ax, 0.0, 0.0, markersize = markersize, color = :green)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)